In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb && make pipeline`
links = pd.read_csv('data/export.csv')
links

/var/folders/3q/_qq6n78n6m92_765npwz342r0000gn/T/ipykernel_40139/964790820.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106354,169988,https://www.jefftk.com/p/post-eag-music-party,Post-EAG Music Party,\N,\N,2023-11-18 16:43:27.459697+00,2023-11-18 16:43:27.459697+00,NaN,\N,JeffKauffman,24.0,0.0,\N
106355,169989,https://www.slowboring.com/p/friday-thread-75d,Friday Thread,\N,\N,2023-11-18 16:43:27.4609+00,2023-11-18 16:43:27.4609+00,NaN,\N,Yglesias,7.0,0.0,\N
106356,169990,https://www.askamanager.org/2023/11/weekend-op...,"weekend open thread – November 18-19\, 2023",\N,www.askamanager.org,2023-11-18 16:43:27.462097+00,2023-11-18 16:47:44.009112+00,0.0,\N,AskManager,90.0,0.0,\N
106357,169991,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2023-11-18 16:43:27.463325+00,2023-11-18 16:43:27.463325+00,NaN,\N,CEA,2.0,0.0,\N


In [2]:
links['added'].max()

'2023-11-18 16:43:27.464518+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299        1306
1300        1307
1301        1308
1302        1309
1303        1310
           ...  
106348    169982
106349    169983
106350    169984
106351    169985
106352    169986
Name: id, Length: 14287, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred


In [5]:
links[links['url'].duplicated()]['id']

Series([], Name: id, dtype: int64)

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'starred': '\\N',
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = pd.concat([links, pd.DataFrame([entry])], ignore_index=True)

links

...1/10
...10/10


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106364,169998,https://twitter.com/austinc3301/status/1726334...,"Agus 🔎 ⏸️~ @ EAGx Virtual on X: ""@peterwildefo...",\N,\N,2023-11-19 21:29:28.844935,2023-11-19 21:29:28.844953,\N,\N,Custom,63.0,0.0,\N
106365,169999,https://twitter.com/notanastronomer/status/172...,"Lauren Gilbert on X: ""since I'm giving a lot o...",\N,\N,2023-11-19 21:29:28.856670,2023-11-19 21:29:28.856681,\N,\N,Custom,62.0,0.0,\N
106366,170000,https://twitter.com/xuanalogue/status/17262459...,"xuan (ɕɥɛn / sh-yen) on X: ""tag yourself I'm 1...",\N,\N,2023-11-19 21:29:28.865819,2023-11-19 21:29:28.865827,\N,\N,Custom,71.0,0.0,\N
106367,170001,https://www.learningfromexamples.com/p/do-most...,Do most people really want to slow down AI? - ...,\N,\N,2023-11-19 21:29:28.874179,2023-11-19 21:29:28.874188,\N,\N,Custom,3.0,0.0,\N


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106364,169998,https://twitter.com/austinc3301/status/1726334...,"Agus 🔎 ⏸️~ @ EAGx Virtual on X: ""@peterwildefo...",\N,\N,2023-11-19 21:29:28.844935,2023-11-19 21:29:28.844953,\N,\N,Custom,63.0,0.0,\N
106365,169999,https://twitter.com/notanastronomer/status/172...,"Lauren Gilbert on X: ""since I'm giving a lot o...",\N,\N,2023-11-19 21:29:28.856670,2023-11-19 21:29:28.856681,\N,\N,Custom,62.0,0.0,\N
106366,170000,https://twitter.com/xuanalogue/status/17262459...,"xuan (ɕɥɛn / sh-yen) on X: ""tag yourself I'm 1...",\N,\N,2023-11-19 21:29:28.865819,2023-11-19 21:29:28.865827,\N,\N,Custom,71.0,0.0,\N
106367,170001,https://www.learningfromexamples.com/p/do-most...,Do most people really want to slow down AI? - ...,\N,\N,2023-11-19 21:29:28.874179,2023-11-19 21:29:28.874188,\N,\N,Custom,3.0,0.0,\N


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106364,169998,https://twitter.com/austinc3301/status/1726334...,"Agus 🔎 ⏸️~ @ EAGx Virtual on X: ""@peterwildefo...",\N,\N,2023-11-19 21:29:28.844935,2023-11-19 21:29:28.844953,\N,\N,Custom,63,0,\N
106365,169999,https://twitter.com/notanastronomer/status/172...,"Lauren Gilbert on X: ""since I'm giving a lot o...",\N,\N,2023-11-19 21:29:28.856670,2023-11-19 21:29:28.856681,\N,\N,Custom,62,0,\N
106366,170000,https://twitter.com/xuanalogue/status/17262459...,"xuan (ɕɥɛn / sh-yen) on X: ""tag yourself I'm 1...",\N,\N,2023-11-19 21:29:28.865819,2023-11-19 21:29:28.865827,\N,\N,Custom,71,0,\N
106367,170001,https://www.learningfromexamples.com/p/do-most...,Do most people really want to slow down AI? - ...,\N,\N,2023-11-19 21:29:28.874179,2023-11-19 21:29:28.874188,\N,\N,Custom,3,0,\N


In [10]:
links.to_csv('data/export.csv', index=False)